This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [95]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [96]:
# First, import the relevant modules
import requests
import json
import numpy as np

In [104]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2019-11-07&end_date=2019-11-07&api_key='+API_KEY)
# 'get' is the request method used to retreive the data
# 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.' portion is the URL for the request
# 'JSON','start_date=2019-11-07&end_date=2019-11-07' are parameters for the format of data retreival and filtering the output
# api_key=API_KEY is the API key with which the user is granted access to the data
print(r.content)

b'{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2019-11-07","end_date":"2019-11-07","frequency":"daily","data":[["2019-11-07",103.7,104.1,101.2,102.2,null,145112.0,14855882.0,null,null,null]],"collapse":null,"order":null}}'


In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [105]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 
#(keep in mind that the date format is YYYY-MM-DD).
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)

In [107]:
#2. Convert the returned JSON object into a Python dictionary.
stock_info_python_dictionary = json.loads(r.content)
# loads method decodes the json object into a python dictionary

In [115]:
# next get the relevant column names and extract the values into a list data
column_names=stock_info_python_dictionary['dataset_data']['column_names']
data = stock_info_python_dictionary['dataset_data']['data']
# define a function for extracting the indices of relevant columns
def get_index(string):
    index=column_names.index(string)
    return index

In [118]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
# use list comprehension to put all the opening price values into a new list open_prices, filter out non-float elements
open_prices = [row[get_index('Open')] for row in data if type(row[get_index('Open')])==float]
print('The highest opening price was '+str(max(open_prices)))
print('The lowest opening price was '+str(min(open_prices)))

The highest opening price was 53.11
The lowest opening price was 34.0


In [117]:
#4. What was the largest change in any one day (based on High and Low price)?
# use list comprehension to make a list of daily changes (the differnce of daily high and low prices)
daily_changes = [row[get_index('High')]-row[get_index('Low')] for row in data]
print('The largest price change in a day was '+str(max(daily_changes)))

The largest price change in a day was 2.8100000000000023


In [119]:
#5. What was the largest change between any two days (based on Closing Price)?
# extract the values of closing prices into a list
close_prices = [row[get_index('Close')] for row in data if type(row[get_index('Close')])==float]
# copy the closing_prices list into a new list, remove the first element and add a dummy value at the end 
# to keep the length of the list constant. Effectively the two lists are now out of register by one element. Taking
# the difference of the two lists creates another list, where each element is the change of closing price between 
# two consecutive days. The last element, the difference between closing price of the last day and the dummy value
# does not represent a price change and is discarded before calculating the maximal change.
close_prices_shifted = close_prices[1:]
close_prices_shifted.append(0.0)
delta_2day=np.array(close_prices)-np.array(close_prices_shifted)
delta_2day=delta_2day[:-1]
print('The largest price change between any two days was ' + str(max(delta_2day)))

The largest price change between any two days was 1.7199999999999989


In [120]:
#6. What was the average daily trading volume during this year?
# extract all the traded volume values into a list
trading_volumes = [row[get_index('Traded Volume')] for row in data]
print('The average daily trading volume was '+str(sum(trading_volumes)/len(trading_volumes)))

The average daily trading volume was 89124.33725490196


In [121]:
#7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own 
#function for calculating the median.)
# define a function for calculating the median value; if the list has an odd number of elements (modulo 2 is 1)
# then the median is the middle element. If the number of elements is even, then median is the average of two 
# middle values. Before performining any calculations the list has to be sorted in ascending order. 
def median_calc(list_vals):
    list_vals.sort()
    if len(list_vals)%2 == 1:
        med_index = int((len(list_vals)-1)/2 + 1)
        median = list_vals[med_index]
    else:
        med_index=len(list_vals)/2
        median = (list_vals[med_index]+list_vals[med_index+1])/2 
    return median
    
print('The median trading volume was '+ str(median_calc(trading_volumes)))

The median trading volume was 76600.0
